In [5]:
import pandas as pd
import requests
import time
import sys
from bs4 import BeautifulSoup as bs
from IPython.core.display import display,HTML
from datetime import datetime
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

pd.set_option('display.max_colwidth', None)

def path_to_image_html(path):
    return '<img src="'+ path + '" width="120" >'

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

/tmp/ipykernel_165/3831264254.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


### Banana Fingers

In [2]:
for i in range(2):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    page = requests.get("https://bananafingers.co.uk/outlet", headers = headers)
    time.sleep(3)
    assert page.status_code == 200

In [3]:
soup = bs(page.text, 'html.parser')
sale = int(soup.findAll('span', class_='toolbar-number')[2].get_text())
pgs = sale/12 if (sale/12)%1 == 0 else int(sale/12)+1
print(f'pages to scan : {pgs}')

pages to scan : 10


In [12]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
for i in [1,100]:
    url = f'https://bananafingers.co.uk/outlet?p={i}'
    page = requests.get(url, headers = headers)
    assert page.status_code == 200
    soup = bs(page.text, 'html.parser')
    if soup.find('div', class_ ='message info empty'):
        print(f'{i} has empty msg')
    else:
        print(f'{i} does not has empty msg')

1 does not has empty msg
100 has empty msg


In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

dict = {'item_name':[],
        'price':[],
        'pecentage_off':[],
        'previous_price':[],
        'img_link':[],
        'item_link':[]}
pg = 1
while True: #exit conditions are a bit complicated so loop infinitely with a break to handle stopping
    url = f'https://bananafingers.co.uk/outlet?p={pg}' # loop through each outlet page
    page = requests.get(url, headers = headers) # call website
    assert page.status_code == 200 # has actually worked
    soup = bs(page.text, 'html.parser') # parse html into text
    if soup.find('div', class_ ='message info empty') or pg >= 100: #stop if the returned html contains an empty warning (ran out of sale items) or pgs >= 100 (just in case)
        break
    all = soup.findAll('div', class_ = 'product-item-info') # find html class for all sale items

    for i in all: # each sale item, get relevant info from html 
        pecentage_off = float(i.find('div', class_ = 'product-labels').get_text().strip().replace('%', ''))
        dict['pecentage_off'].append(pecentage_off)
        dict['img_link'].append( i.find('img')['src'])
        dict['item_name'].append(i.find(class_='product-item-link').get_text().strip())
        dict['item_link'].append( i.find(class_='product-item-link')['href'])
        price = float(i.find(class_='price').get_text().strip('£'))
        dict['price'].append(price)
        dict['previous_price'].append(price / (1-(pecentage_off/100)))
    
    print(f'page {pg} finished at {datetime.now().strftime("%H:%M:%S")}') # checking
    pg += 1 next page


page 1 finished at 00:54:54
page 2 finished at 00:54:54
page 3 finished at 00:54:55
page 4 finished at 00:54:55


### Rockrun

In [5]:
opts = FirefoxOptions()
opts.add_argument("--headless")
browser = webdriver.Firefox(options=opts)

browser.get('https://rockrun.com/collections/all-deals')
time.sleep(1)

body = browser.find_element(By.CSS_SELECTOR, "body")
no_of_pagedowns = 50

while no_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1

soup = bs(browser.page_source)
all = soup.findAll('div', class_='product-wrap')

for i in all:
    dict['item_name'].append(i.find(class_ ='product-thumbnail__title').get_text())
    price = float(i.find(class_ = 'money').get_text().strip().replace('£',''))
    dict['price'].append(price)
    previous_price = price if i.find(class_ = 'product-thumbnail__was-price compare-at-price') is None else float(i.find(class_ = 'product-thumbnail__was-price compare-at-price').get_text().strip().replace('£',''))
    dict['previous_price'].append(previous_price)
    dict['pecentage_off'].append((1 - (price/previous_price))*100)
    dict['item_link'].append(f"https://rockrun.com{i.find('a')['href']}")
    dict['img_link'].append(f"https://{i.find('img')['src'].strip('/')}")

La Sportiva Kubo (Sizes EU35-39.5)
Moon Dust Climbing Chalk (300g) 2-Pack
Prana Kragg Pant - Mens
DMM XSRE Black 3-Pack
Blue Ice Moonlight 35L (2023)
Prana Mojo Short - Men's (Admiral Blue)
Deuter Guide Lite 24 (2022)
La Sportiva Travel Bag
Edelrid Swift 48 Pro Dry 8.9mm x 50m
Edelweiss Discover 8 Walking Rope
Blue Ice Cuesta Womens Harness Package
Blue Ice Cuesta Mens Harness Package
Scarpa Vapour V (2024)
Wild Country Superlight Offset Rocks Set 5-10
La Sportiva Trango Tech Leather GTX
DMM Pitcher Rope Bag
Scarpa Instinct Lace Womens
La Sportiva TX2 Evo
La Sportiva Tempesta GTX
Boreal Ninja
Scarpa Boostic
EB Red
Moon Samurai Pant - Women's
Petzl ANGE L
Black Diamond Forged Denim Pants - Women's
EB Black Opium
Scarpa Mescalito Planet
Black Diamond Notion SP Pants (2023) - Women's
Petzl Luna Womens Harness (2024)
Petzl Adjama Harness (2024)
Metolius Campus Blocks
Blue Ice Choucas Pro Harness
Black Diamond Cirque Gaiter
La Sportiva TX2 Evo Leather - Womens
Black Diamond Mission Leather 

### Output Dataframe

In [6]:
new = pd.DataFrame(dict)
new.sort_values('pecentage_off', inplace =True, ascending = False)
new.reset_index(drop=True, inplace = True)
new['previous_price'] = new['previous_price'].apply(lambda x: "£{:.2f}".format((x)))
new['price'] = new['price'].apply(lambda x: "£{:.2f}".format((x)))
new['pecentage_off'] = new['pecentage_off'].apply(lambda x: "{:.0f}%".format(x))

In [7]:
format_dict = {'img_link':path_to_image_html, 'item_link':make_clickable}

In [8]:
show = min(50, len(new))
display(HTML(new.head(show).to_html(escape=False, formatters=format_dict, index = False)))

item_name,price,pecentage_off,previous_price,img_link,item_link
Exped Schnozzel Pumpbag UL L,£9.95,73%,£37.50,,https://rockrun.com/collections/all-deals/products/exped-schnozzel-pumpbag-ul-l
Moore's Wall Bouldering (North Carolina),£11.40,70%,£38.00,,https://rockrun.com/collections/all-deals/products/moores-wall-bouldering
Evolv Geshido,£55.00,66%,£161.76,,https://bananafingers.co.uk/evolv-geshido
Looking For Wild Bavella Shorts - Womens (Navy Peony),£24.95,62%,£65.00,,https://rockrun.com/collections/all-deals/products/looking-for-wild-bavella-shorts-womens-navy-peony
Black Diamond Talus Tank - Women's (Size XL),£21.95,60%,£55.00,,https://rockrun.com/collections/all-deals/products/black-diamond-talus-tank-womens
Black Diamond Notion Shorts - Women's (Size L),£23.95,60%,£60.00,,https://rockrun.com/collections/all-deals/products/black-diamond-notion-shorts-womens
Outdoor Research Shastin Pant - Tapenade - Women's,£30.95,59%,£75.00,,https://rockrun.com/collections/all-deals/products/outdoor-research-shastin-pant-womens
EB Black Opium,£49.95,57%,£115.00,,https://rockrun.com/collections/all-deals/products/eb-black-opium
La Sportiva TX2 Evo,£69.95,56%,£160.00,,https://rockrun.com/collections/all-deals/products/la-sportiva-tx2-evo
Prana Kragg Pant - Mens,£40.00,56%,£90.00,,https://rockrun.com/collections/all-deals/products/prana-kragg-pant-mens
